# Pipeline for analysis of Galileo/SSI images with photometric models

## 1. Downloading Galileo/SSI images from Pilot

Image search tool: https://pilot.wr.usgs.gov/

Constraints:

1. Reasonable image resolution that provides large variation in emission/incidence angles for roughness modeling (between 0.5 to 5 km/pix should do)  
2. Coverage of entire globe with minimal overlaps
3. Low phase angle observations (<70 deg.), where the effect of multiple scattering on dilution of shadows is minimal (Buratti & Veverka (1985)), Domingue et al. (1991), Domingue and Hapke (1992))
4. Visible wavelength filters. Pretty much all shortlisted images were taken with clear filter. IR stuff is beyond the scope of this study.

**Note**: For each image, make sure to download both the **.img** and **.lbl** extension files. One contains the data and one contains important header info. The ISIS commands described below won't work without both files. 

## 2. Generate photometric cubes using ISIS

1. Install and set up ISIS: https://github.com/DOI-USGS/ISIS3#installation
2. Need to download mission specific data so that we have the right kernels to do all the observation geometry calculations: https://astrogeology.usgs.gov/docs/how-to-guides/environment-setup-and-maintenance/isis-data-area/

**Note:** After a lot of trial-and-error, I found that **ISIS 8** and **Python 3.9** is a stable combo. This might have changed in the last couple of years though.


In a folder full of .img and .lbl files for the shortlisted image, I ran the following bash script to generate the photometric cube files (with custom extensions of **.pho.cub**):


```bash
#!/bin/bash

# Define input and output directories
INPUT_DIR="All data files"
OUTPUT_DIR="All data files/Post ISIS"

# Make sure output directory exists
mkdir -p "$OUTPUT_DIR"

# Process each .lbl file in the input directory
for file in "$INPUT_DIR"/*.lbl; do
    # Get base name without path and extension
    base_name=$(basename "$file" .lbl)

    echo "Processing $base_name..."

    # Full paths for intermediate and output files
    cub_file="$OUTPUT_DIR/${base_name}.cub"
    cal_cub_file="$OUTPUT_DIR/${base_name}_cal.cub"
    pho_cub_file="$OUTPUT_DIR/${base_name}_cal.pho.cub"

    # Run ISIS processing
    gllssi2isis FROM="$file" TO="$cub_file"
    spiceinit FROM="$cub_file" WEB=false
    gllssical FROM="$cub_file" TO="$cal_cub_file"
    phocube FROM="$cal_cub_file" TO="$pho_cub_file" DN=yes PHASE=yes EMISSION=yes INCIDENCE=yes LATITUDE=yes LONGITUDE=yes PIXELRESOLUTION=yes
done

echo "Processing complete."

See example files in the `Image Files` folder associated with the galileo image `5126r`. The footer label of the .pho.cub files lists all the commands used to generate the .pho.cub file

## 3. Use `galileo-ssi` to analyze the photometric cubes and extract terrain-specific data

Basic data file walk-through using data_exploration_example.ipynb

Use plotting script from full_pipeline.ipynb

## 4. Extract photometric properties

### 4.1. Crater reflectance model (Buratti & Veverka, 1985)

### 4.2. Hapke photometric model 